## Beta-2 adrenergic receptor - part 2 (MMP identification)

### Import libraries

In [1]:
import pandas as pd

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

import sys
sys.path.append('/home/daniel/wizepair2')
from classes.mmp import MMP

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Read in the dataset

In [3]:
df = pd.read_csv('beta2_agonists_stripped.csv')
len(df.index)

1223

### Create cartesian product of unique molecules tested in the same assay

In [4]:
df_pairs = pd.merge(df, df, on=['target_pref_name', 'standard_type'])
df_pairs = df_pairs[['stripped_smiles_x', 'stripped_smiles_y']].drop_duplicates()
df_pairs

,stripped_smiles_x,stripped_smiles_y
0,COc1ccccc1OCCNC[C@@H](O)c1ccc(O)c(O)c1,COc1ccccc1OCCNC[C@@H](O)c1ccc(O)c(O)c1
1,COc1ccccc1OCCNC[C@@H](O)c1ccc(O)c(O)c1,CNC[C@H](O)c1ccc(O)c(O)c1F
2,COc1ccccc1OCCNC[C@@H](O)c1ccc(O)c(O)c1,CNCC(O)c1ccc(O)c2nc(O)sc12
3,COc1ccccc1OCCNC[C@@H](O)c1ccc(O)c(O)c1,CNCC(O)c1cc(O)c(O)cc1F
4,COc1ccccc1OCCNC[C@@H](O)c1ccc(O)c(O)c1,CC(C)NCC(O)c1ccc(Cl)c(Cl)c1
...,...,...
31395,CCC(c1ccc(O)c(O)c1)C(CC)c1ccc(O)c(O)c1,CCC(c1ccc(O)c(O)c1)C(CC)c1ccc(O)c(O)c1
31434,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1,CC(C)NC[C@H](O)c1ccc2ccccc2c1
31439,CC(C)NC[C@H](O)c1ccc2ccccc2c1,CC(C)NC[C@@H](O)c1ccc(O)c(O)c1
31442,CC(C)NC[C@H](O)c1ccc2ccccc2c1,CNCC(O)c1ccc(O)c(O)c1


### Identify all pairs

In [5]:
df_pairs = df_pairs.sample(frac=1).parallel_apply(lambda x: MMP(x.stripped_smiles_x, x.stripped_smiles_y, strictness=5, correspondence=2).execute(), axis=1)

In [6]:
df_pairs = pd.json_normalize(df_pairs.explode())
df_pairs.sample(3).transpose()

,4643,2248,1991
smiles1,CC(C)(C)NCC(O)c1ccc(O)cc1,CNC[C@H](O)c1cccc(O)c1,CC(C)(C)NCC(O)c1ccc(O)c(O)c1
smiles2,CC(C)NCC(O)c1ccc(O)c(O)c1,CNCCc1ccc(O)c(O)c1,CNC[C@H](O)c1ccc(O)c(O)c1
percentmcs,0.8,0.692308,0.6875
radius,4.0,2.0,1.0
valid,True,True,True
solversecs,0.02505,0.005501,0.025315
embedding,"[26, 24, 31, 5, 6, 25, 0, 0, 0, 0, 0, 0, 0, 0,...","[32, 4, 12, 12, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[23, 31, 9, 8, 23, 2, 0, 0, 0, 0, 0, 0, 0, 0, ..."
predsolversecs,0.242788,0.103588,0.184302
error,None,None,None
smirks,[#1]-[#6](-[#1])(-[#1])-[#6:7](-[#7:6](-[#1])-...,[#1]-[#8]-[#6@@](-[#1])(-[#6:1](-[#1])(-[#1])-...,[#1]-[#6](-[#8:1]-[#1])(-[#6:2](-[#1])(-[#1])-...


### Drop failures and write output to file

In [7]:
df_pairs = df_pairs[df_pairs.valid]
df_pairs.to_csv('beta2_agonists_pairs.csv', index=False)